[DigiPath_MLTK functions](https://dlanier.github.io/DigiPath_MLTK/) <br>

# Low Level functionality for getting patches per image level

### *patch_location_array = get_patch_location_array_for_image_level(run_parameters)*
```python
# help(get_patch_location_array_for_image_level)

""" Usage: 
patch_location_array = get_patch_location_array_for_image_level(run_parameters)
                        using patch_select_method, find all upper left corner locations of patches
                        that won't exceed image size givin the 'patch_height' and 'patch_width

Args (run_parameters):  python dict.keys()
                            wsi_filename:           file name (with valid path)
                            patch_height:           patch size = (patch_width, patch_height)
                            patch_width:            patch size = (patch_width, patch_height)
                            thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                            patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                            threshold:              minimimum sum of thresholded image (default = 0)
                            image_level:            openslide image pyramid level 0,1,2,...
Returns:
    patch_location_array:   [[x, y], [x, y],... ]   n_pairs x 2 numpy array
"""
```
### *mask_im, prev_im, patch_array = get_patch_locations_preview_image_for_image_level(run_parameters)*
```python
# help(get_patch_locations_preview_image_for_image_level)

""" Usage: with run_parameter dict "run_pars"
mask_image, thumb_preview, patch_location_array = get_patch_locations_preview_image_for_image_level(run_pars)
                                                  create viewable images to show patch locations

Args (run_parameters):  python dict.keys()
                            wsi_filename:           file name (with valid path)
                            patch_height:           patch size = (patch_width, patch_height)
                            patch_width:            patch size = (patch_width, patch_height)
                            thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                            patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                            threshold:              sum of thresholded image minimimum (default = 0)
                            image_level:            openslide image pyramid level 0,1,2,...

                        Optional keys()
                            border_color:           patch-box representation color red, blue, green, ...

Returns:
    mask_image:             black & white image of the mask
    thumb_preview:          thumbnail image with patch locations marked
    patch_location_array:   list of patch locations used [(row, col), (row, col),... ]
"""
```

In [ ]:
import time
note_book_cell_seq_run_time = time.time()
import os
import sys

from pychunklbl.toolkit import get_patch_location_array_for_image_level
from pychunklbl.toolkit import get_patch_locations_preview_image_for_image_level
from pychunklbl.toolkit import get_file_size_ordered_dict, lineprint_level_sizes_dict, get_level_sizes_dict
DEFAULT_THUMBNAIL_DIVISOR = 20

data_dir = '../../../ncsa/DigiPath_MLTK_data/Aperio'

file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

In [ ]:
#                         test data file for threshold_otsu
image_file_name = 'CMU-2.svs'
# image_file_name = 'JP2K-33003-2.svs'
# image_file_name = 'CMU-1-Small-Region.svs'

run_parameters = dict()
run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = DEFAULT_THUMBNAIL_DIVISOR
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224
run_parameters['threshold'] = 0
run_parameters['border_color'] = 'blue'

lineprint_level_sizes_dict(run_parameters['wsi_filename'])

In [ ]:
level_sizes_dict = get_level_sizes_dict(run_parameters['wsi_filename'])

n_levels = level_sizes_dict['level_count']
for im_lvl in range(0, n_levels):
    run_parameters['image_level'] = im_lvl
    run_parameters['thumbnail_divisor'] = DEFAULT_THUMBNAIL_DIVISOR
    
    t1 = time.time()

    patch_location_array = get_patch_location_array_for_image_level(run_parameters)
    print('\nimage_level = %i,\nnumber of patches = %i'%(im_lvl, len(patch_location_array)))
    
    thmb_div_4_level = run_parameters['thumbnail_divisor'] // (2*run_parameters['image_level'] + 1)
    run_parameters['thumbnail_divisor'] = thmb_div_4_level
    
    mask_im, prev_im, patch_array = get_patch_locations_preview_image_for_image_level(run_parameters)

    print('image_level = %i,\nnumber of patches = %i'%(im_lvl, len(patch_array)))
    print('cell run time: %0.3f'%(time.time() - t1))
    print('thumb image size:', prev_im.size, 'thumbnail_divisor', run_parameters['thumbnail_divisor'])
    
    display(prev_im)

### Help on utility functions used for notebook organization and display:

In [ ]:
help(get_level_sizes_dict)

In [ ]:
help(lineprint_level_sizes_dict)

In [ ]:
help(get_file_size_ordered_dict)

In [ ]:
print('Notebook total run time = %0.3f seconds'%(time.time() - note_book_cell_seq_run_time))